Train MLP for each finger pair!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne
from random import shuffle

mne.set_log_level('error')


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
subject_id = 0

In [3]:
# Load the dictionary from the HDF5 file
target_dir = 'features'
file_path = os.path.join(target_dir, cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 6732)
little (50, 6732)
middle (50, 6732)
ring (50, 6732)
thumb (50, 6732)


In [4]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

In [5]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test, y_pred)
    return acc

def objective(trial, X, y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 2, activation)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)

    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    # fold_accuracies = []

    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X[train_index], X[test_index]
    #     y_train, y_test = y[train_index], y[test_index]

    #     model = SingleLayerMLP(X_train.shape[1], hidden_size, 2, activation)
    #     criterion = nn.CrossEntropyLoss()
    #     optimizer = optimizer(model.parameters(), lr=learning_rate)
    #     acc = train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=num_epochs)
    #     fold_accuracies.append(acc)

    # mean_accuracy = np.mean(fold_accuracies)
    # return mean_accuracy

def train_MLP(finger1, finger2, verbose = True):
   
    print(f'Training MLP for {finger1} vs {finger2}')

    X = np.concatenate((data[finger1], data[finger2]), axis=0)
    y = np.concatenate((np.ones(data[finger1].shape[0]), np.zeros(data[finger2].shape[0])), axis=0)

    X,y = shuffle(X,y)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=10)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')



In [6]:
train_MLP('middle', 'ring', verbose = True)

Training MLP for middle vs ring


TypeError: Random.shuffle() takes 2 positional arguments but 3 were given

In [ ]:
for finger1 in data:
    for finger2 in data:
        if finger1 != finger2:
            train_MLP(finger1, finger2)
        else:
            break

[I 2023-04-14 14:42:12,737] A new study created in memory with name: no-name-5a83cdb7-bcfd-46d5-aeae-191b0479f31b


Training MLP for little vs index


[I 2023-04-14 14:42:16,799] Trial 0 finished with value: 0.35 and parameters: {'learning_rate': 0.019038652445492337, 'num_epochs': 1867, 'hidden_size': 24, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.35.
[I 2023-04-14 14:42:19,418] Trial 1 finished with value: 0.4 and parameters: {'learning_rate': 0.005486859007369514, 'num_epochs': 1264, 'hidden_size': 53, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.4.
[I 2023-04-14 14:42:20,172] Trial 2 finished with value: 0.5 and parameters: {'learning_rate': 0.0030136666718723693, 'num_epochs': 441, 'hidden_size': 29, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 2 with value: 0.5.
[I 2023-04-14 14:42:27,138] Trial 3 finished with value: 0.55 and parameters: {'learning_rate': 0.0029383246394954587, 'num_epochs': 960, 'hidden_size': 100, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 3 with value: 0.55.
[I 2023-04-14 14:42:29,922] Trial 4 finished with value: 0.4

Best trial params: {'learning_rate': 0.0029383246394954587, 'num_epochs': 960, 'hidden_size': 100, 'activation': 'leaky_relu', 'optimizer': 'Adam'}
Best trial accuracy: 55.00%
Training MLP for middle vs index


[I 2023-04-14 14:42:49,428] Trial 0 finished with value: 0.65 and parameters: {'learning_rate': 0.0008488758543516051, 'num_epochs': 1011, 'hidden_size': 90, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.65.
[I 2023-04-14 14:42:49,860] Trial 1 finished with value: 0.6 and parameters: {'learning_rate': 0.0035946072968360394, 'num_epochs': 217, 'hidden_size': 56, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.65.
[I 2023-04-14 14:42:54,875] Trial 2 finished with value: 0.7 and parameters: {'learning_rate': 0.0002462211311570272, 'num_epochs': 1970, 'hidden_size': 93, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 2 with value: 0.7.
[I 2023-04-14 14:42:57,177] Trial 3 finished with value: 0.6 and parameters: {'learning_rate': 0.010647837456255882, 'num_epochs': 888, 'hidden_size': 84, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 2 with value: 0.7.
[I 2023-04-14 14:42:58,526] Trial 4 finished with value:

...

Best trial params: {'learning_rate': 0.02343595414612765, 'num_epochs': 414, 'hidden_size': 114, 'activation': 'leaky_relu', 'optimizer': 'SGD'}
Best trial accuracy: 70.00%
Training MLP for thumb vs middle


[I 2023-04-14 14:47:37,798] Trial 0 finished with value: 0.45 and parameters: {'learning_rate': 3.1395805544958564e-05, 'num_epochs': 619, 'hidden_size': 18, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.45.
[I 2023-04-14 14:47:45,253] Trial 1 finished with value: 0.65 and parameters: {'learning_rate': 7.017864767254146e-05, 'num_epochs': 1984, 'hidden_size': 93, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.65.
[I 2023-04-14 14:47:48,656] Trial 2 finished with value: 0.55 and parameters: {'learning_rate': 0.00017904659762582812, 'num_epochs': 677, 'hidden_size': 67, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.65.
[I 2023-04-14 14:47:55,909] Trial 3 finished with value: 0.6 and parameters: {'learning_rate': 0.0044678450820598405, 'num_epochs': 1025, 'hidden_size': 119, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.65.
[I 2023-04-14 14:47:58,335] Trial 4 finished w

Best trial params: {'learning_rate': 7.017864767254146e-05, 'num_epochs': 1984, 'hidden_size': 93, 'activation': 'relu', 'optimizer': 'SGD'}
Best trial accuracy: 65.00%
Training MLP for thumb vs ring


[I 2023-04-14 14:48:18,114] Trial 0 finished with value: 0.8 and parameters: {'learning_rate': 6.319206534323966e-05, 'num_epochs': 1751, 'hidden_size': 49, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8.
[I 2023-04-14 14:48:23,945] Trial 1 finished with value: 0.9 and parameters: {'learning_rate': 0.048412582012581404, 'num_epochs': 1085, 'hidden_size': 59, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.9.
[I 2023-04-14 14:48:28,146] Trial 2 finished with value: 0.9 and parameters: {'learning_rate': 0.00039279527022275214, 'num_epochs': 1458, 'hidden_size': 105, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 1 with value: 0.9.
[I 2023-04-14 14:48:29,254] Trial 3 finished with value: 0.85 and parameters: {'learning_rate': 0.017905221714138483, 'num_epochs': 166, 'hidden_size': 96, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.9.
[I 2023-04-14 14:48:32,015] Trial 4 finished with val

Best trial params: {'learning_rate': 6.279837210346075e-05, 'num_epochs': 1510, 'hidden_size': 21, 'activation': 'elu', 'optimizer': 'SGD'}
Best trial accuracy: 95.00%
